In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

In [2]:
import warnings

warnings.filterwarnings('ignore')

#### Для любой модели из курса(кроме KNN) и данных на которых она строилась: примнить метод главных компонент к данным, и сравнить работу модели на исходных данных с работой на данных где в качестве признаков используются две главные компоненты.

## Результат для адаптивного бустинга с логистической регрессией

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from sklearn import model_selection

import numpy as np
import warnings

In [4]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

def get_error(pred, y):
    return np.sum((pred != y).astype(int)) / len(y)

def log_loss(pred, y): 
    return -np.sum(y*np.log2(pred)+(1-y)*np.log2(1-pred))/len(y)

In [5]:
def adaboost(X, y, N):

    # Запишем количество классов в переменную
    n_classes = len(np.unique((y)))

    models = []

    for n in range(N):
        clf = LogisticRegression(random_state=0)
        #clf.fit(X, y, w)
        clf.fit(X, y)
        
        predictions = clf.predict(X)
        e = log_loss(predictions, y)
        
        alpha = 0.5 * np.log((1 - e) / e)

        match = predictions == y

        models.append((alpha, clf))
    
    return models

In [6]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    N = 50
    
    models = adaboost(X_train, y_train, N)

In [7]:
def predict(X, models):
    classes = []
    for alpha, clf in models:
        prediction = clf.predict(X)
        classes.append(prediction)
        
    return np.array(classes)

print(f'Точность алгоритма на обучающей выборке: {(100 - get_error(predict(X_train, models), y_train)):.3f}')

Точность алгоритма на обучающей выборке: 98.122


In [8]:
print(f'Точность алгоритма на тестовой выборке: {(100 - get_error(predict(X_test, models), y_test)):.3f}')

Точность алгоритма на тестовой выборке: 96.853


## Применение PCA для адаптивного бустинга с логистической регрессией

In [12]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

In [13]:
# Для начала отмасштабируем выборку
X = X_train.astype(float)
mean=np.apply_along_axis(np.mean, 0, X)
std=np.apply_along_axis(np.std, 0, X)
X_normed_trained=(X-mean[np.newaxis, :])/std[np.newaxis, :]

In [14]:
# Найдем собственные векторы и собственные значения
 
covariance_matrix = (X_normed_trained.T).dot(X_normed_trained)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений

sorted(eig_pairs, reverse=True)

# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack((eig_pairs[0][1].reshape(30,1), eig_pairs[1][1].reshape(30,1)))

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.22036826 -0.23166116]
 [ 0.10720321 -0.08157197]
 [ 0.22853368 -0.21336647]
 [ 0.22230466 -0.22761029]
 [ 0.1457819   0.19535568]
 [ 0.24086238  0.14772976]
 [ 0.25833871  0.0627911 ]
 [ 0.2607821  -0.0327151 ]
 [ 0.13970422  0.18331767]
 [ 0.06113533  0.37323648]
 [ 0.20267142 -0.10529754]
 [ 0.01898002  0.07259228]
 [ 0.20772676 -0.08814432]
 [ 0.20029222 -0.1493035 ]
 [ 0.01868004  0.20367987]
 [ 0.17472731  0.23265899]
 [ 0.14795385  0.20617466]
 [ 0.18284622  0.1397113 ]
 [ 0.02617093  0.1556857 ]
 [ 0.10217937  0.29283626]
 [ 0.22898972 -0.21780584]
 [ 0.10926407 -0.06383144]
 [ 0.23746601 -0.19733983]
 [ 0.22645776 -0.21580411]
 [ 0.13067574  0.17738698]
 [ 0.2106612   0.13664677]
 [ 0.22704443  0.10187933]
 [ 0.25003448 -0.00311742]
 [ 0.12031118  0.12015367]
 [ 0.13021554  0.27998338]]


In [15]:
new_X_train=X_normed_trained.dot(W)

In [16]:
X_normed_test=(X_test-mean[np.newaxis, :])/std[np.newaxis, :]
new_X_test=X_normed_test.dot(W)

#new_X_test

In [17]:
N = 50
    
models = adaboost(new_X_train, y_train, N)

In [18]:
print(f'Точность алгоритма на обучающей выборке: {(100 - get_error(predict(new_X_train, models), y_train)):.3f}')

Точность алгоритма на обучающей выборке: 98.239


In [19]:
print(f'Точность алгоритма на тестовой выборке: {(100 - get_error(predict(new_X_test, models), y_test)):.3f}')

Точность алгоритма на тестовой выборке: 97.552
